# Loading and modifying a SMIRNOFF forcefield

This notebook illustrates how to load a SMIRNOFF forcefield, apply it to an example molecule, get the energy, then manipulate the parameters in the forcefield and update the energy.

## Prep some utility functions/import stuff

In [5]:
# Imports needed
from openforcefield.typing.engines.smirnoff.forcefield import ForceField, generateTopologyFromOEMol
import openeye
from openeye import oechem
from openforcefield.utils import get_data_filename
from simtk import openmm
from simtk import unit
import numpy as np

In [6]:
# Define utility function we'll use to get energy of an OpenMM system
def get_energy(system, positions):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
    Returns
    ---------
    energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    return energy

## Load an OEMol, evaluate energy before and after a parameter modification

In [7]:
# Load simple OEMol
verbose = False
# Load one of the provided files
ifs = oechem.oemolistream(get_data_filename('molecules/AlkEthOH_c100.mol2'))
mol = oechem.OEMol()
# This uses parm@frosst atom types, so make sure to use the forcefield-flavor reader
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
# Perceive tripos types
oechem.OETriposAtomNames(mol)

#Get positions for use below
coordinates = mol.GetCoords()
natoms=len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, unit.angstroms)

# Load forcefield file
ffxml = get_data_filename('forcefield/Frosst_AlkEthOH.ffxml')
ff = ForceField(ffxml)

# Generate a topology
topology = generateTopologyFromOEMol(mol)

# Create initial system
system = ff.createSystem(topology, [mol], verbose=verbose)

# Get initial energy before parameter modification
old_energy=get_energy(system, positions)

# Get params for an angle
params = ff.getParameter(smirks='[a,A:1]-[#6X4:2]-[a,A:3]')
# Modify params
params['k']='0.0'
ff.setParameter(params, smirks='[a,A:1]-[#6X4:2]-[a,A:3]')

# Evaluate energy after parameter modification
system=ff.createSystem(topology, [mol], verbose=verbose)
energy=get_energy(system, positions)

# Print out energy
print("Original energy: %.3g. New energy: %.3g" % (old_energy, energy))

Original energy: -20.9. New energy: -22.1


## Take a look at what other info is provided on parameters and how to access it

In [8]:
# Load forcefield file
ffxml = get_data_filename('forcefield/Frosst_AlkEthOH.ffxml')
ff = ForceField(ffxml)

# Get a parameter by parameter id
param = ff.getParameter(paramID='b0001')
print(param)

# Get a parameter with a search restricted to a particular section, by smirks
param = ff.getParameter(smirks='[$([#1]-C):1]', force_type='NonbondedForce')
print(param)

{'smirks': '[#6X4:1]-[#6X4:2]', 'length': '1.526', 'k': '620.0', 'id': 'b0001', 'parent_id': 'b0001'}
None
